In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from llama_index.core import Document

In [7]:
## config

from pydantic import Field
from pydantic_settings import BaseSettings
import os
from dotenv import load_dotenv


class Settings(BaseSettings):
    model_name: str = Field("gigachat")
    api_key: str = Field(...)
    
    top_k_bm: int = Field(20)
    top_k_emb: int = Field(20)

    class Config:
        env_file = ".env"


settings = Settings()

In [5]:
from langchain.chat_models.gigachat import GigaChat

In [ ]:
model = GigaChat(
    verify_ssl_certs=False, 
    credentials="api_key",
    model="GigaChat:latest"
)

In [7]:
model.invoke('hello')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': Usage(prompt_tokens=12, completion_tokens=10, total_tokens=22), 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'stop'}, id='run-5bcf023a-6f0f-426c-814f-49ad4a4b7548-0')

In [1]:
import sys
sys.path.append("../")

In [2]:
from utils.llms.GigaModel import GigaApi

In [3]:
model = GigaApi()

c:\Мисис\python_ml\notebooks\..\utils\llms\GigaModel.py:26: LangChainDeprecationWarning: The class `GigaChat` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-gigachat package and should be used instead. To use it run `pip install -U :class:`~langchain-gigachat` and import as `from :class:`~langchain_gigachat import GigaChat``.
  self.chat = GigaChat(model="GigaChat:latest", credentials=self.credentials, verify_ssl_certs=False)


In [4]:
model.inference('hello')

'Hello! How can I assist you today?'

# Parsing

In [43]:
from dataclasses import dataclass
import PyPDF2
import re

def read_pdf_pages(file_path):
    text = []
    with open(file_path, 'rb') as f:
        pdfReader = PyPDF2.PdfReader(f)
        count = len(pdfReader.pages)
        for i in range(count):
            page = pdfReader.pages[i]
            output = page.extract_text()
            text.append(output)
    return text


def parse_passage_num(line):
    num = -1
    gr1 = re.search(r'^\s*(Статья\s+\d+(\.\d+)*[абвгдежзик]?|\d+(\.\d+)*[абвгдежзик]?|[абвгдежзик]\))', line)
    if gr1:
        num = gr1.group(0).strip()
        # Проверить не является ли датой
        if re.search('^\d{2}.\d{2}.\d{4}', line.strip()):
            num = -1
        # Проверить есть ли слово после номера
#         if ' ' not in line or len(line.split(' ', 1)[1]) < 6:
#             num = -1
        # Пассажи всегда начинаются с большой буквы кроме случаев 1) а)
        if not line.strip().startswith('Статья') and not line.strip().split()[0].endswith(')'):
            if len(line.strip().split()) > 1 and not line.strip().split()[1][0].isupper():
                num = -1

        # Когда после номера статьи должна идти точка и большая буква
        elif line.strip().startswith('Статья') and len(line.strip().split(' ', 2)) > 2:
            if line.strip().split(' ', 2)[2][0].islower():
                num = -1

        # Сохранение случаев "1)"
        tmp = str(num) + ')'
        if line.startswith(tmp):
            num = tmp

    return num

@dataclass
class Passage:
    text: str = ''
    passage_number: str = None
    page_number_first: int = None
    page_number_last: int = None
    text_with_meta: str = ''


class NPA:
    def __init__(self, file_path, passage_len_limits=[10, 8000]):
#         self.doc_type = doc_type
        self.doc_text = read_pdf_pages(file_path)
        self.doc_name = file_path.split('/')[-1]
        self.preproc_npa_text()
        self.passage_len_limits = passage_len_limits
        self.extract_passages()
        self.add_meta_info_passages()

    def preproc_npa_text(self):
        # Вырезает шапку и ставляет номера страниц в нужном формате: PAGE=i
        if 'ГАРАНТ' in self.doc_text[0]:
            text = [re.split(r'Система\s*[ГАРАНТ\s]*\s*\d+/\d+', p)[1].strip() for p in self.doc_text]
        else:
            text = [re.split(r'Страница\s*\d+\s*из\s*\d+', p)[0].strip() if
                        len(re.findall(r'Страница\s*\d+\s*из\s*\d+', p)) > 0 else '' for p in self.doc_text]
        text_plain = ''
        for i in range(len(text)):
            text_plain += f'\nPAGE={i+1}\n' + text[i]
        text_plain = re.sub(r'\n ', '\n', text_plain)
        text_plain = text_plain.replace('www.consultant.ru', '')
        # text_plain = text_plain.replace(' КонсультантПлюс надежная правовая поддержка', '')
        # text_plain = re.sub("КонсультантПлюс надежная правовая поддержка", '', text_plain)
        self.doc_text = text_plain

    def extract_passages(self):
        lines = self.doc_text.split('\n')
        passages = []
        cur_passage = Passage()
        page_num = None

        for line in lines:
            # Определить номер текущей страницы
            page_nums = re.findall(r'PAGE=(\d+)', line)

            if page_nums:
                page_num = int(page_nums[0])

            line_text = line.strip()
            num = parse_passage_num(line_text)

            # Были случаи где Статья и ее номер были разделены двумя пробелами
            if 'Статья' in str(num):
                lst = num.split()
                num = lst[0] + ' ' + lst[1]

            # Новый блок документа -> сбрасываем стейт (см дальше при добавлении пассажа)
            if num != -1:
                if cur_passage.text:
                    passages.append(cur_passage)
                cur_passage = Passage(text=line_text, passage_number=num,
                                      page_number_first=page_num, page_number_last=page_num)
            else:
                if cur_passage.passage_number is not None:
                    if cur_passage.text or line_text:
                        cur_passage.text += '\n' + line_text
                        cur_passage.page_number_last = page_num

        if cur_passage.text:
            passages.append(cur_passage)


        # passage text postproc
        for p in passages:
            p.text = re.sub(r'PAGE=\d+', '', p.text)
            p.text = re.sub(r'(\D)\n+(\D)', r'\1 \2', p.text.strip())

        # отфильтровать по длине пассажа
        self.passages = [p for p in passages if self.passage_len_limits[0] <= len(p.text) <= self.passage_len_limits[1]]


    def add_meta_info_passages(self):
        if self.passages is None:
            self.extract_passages()

        # Добавить пассажи первого уровня ко вторым и наоборот
        first_level_text = ''
        first_level_idx = None
        for j, p in enumerate(self.passages):
            # TO FIX: If Статья -> Статья is first level, else ...
            if p.passage_number.startswith('Статья'):
                first_level_text = p.text
                p.text_with_meta = p.text
                first_level_idx = j

            elif not first_level_text.startswith('Статья') and ('.' not in p.passage_number and ')' not in p.passage_number):
                first_level_text = p.text
                p.text_with_meta = p.text
                first_level_idx = j

            else:
                p.text_with_meta = first_level_text + '\n' + p.text
                if first_level_idx is not None:
                    self.passages[first_level_idx].text_with_meta += '\n' + p.text
                    self.passages[first_level_idx].page_number_last = p.page_number_last



In [44]:
import os
passages_df_full = []
file_names = ['44_FZ.pdf', '223_FZ.pdf']
data_path_npa_pdf = r'../data'

for name in file_names:
    npa = NPA(rf"{os.path.join(data_path_npa_pdf, name)}")
    passages_npa_df = []

    for p in npa.passages:
        passages_npa_df.append(
            {
                'passage': p.text,
                'passage_full': p.text_with_meta,
                'number': p.passage_number,
                'page_first': p.page_number_first,
                'page_last': p.page_number_last,
                'title': name,
            }
        )
    passages_df_full.extend(passages_npa_df)
    print(f'add {name} to dataframe')


temp = pd.DataFrame(passages_df_full)

temp['passage'] = temp['passage'].apply(lambda x: x.replace('КонсультантПлюс надежная правовая поддержка', ''))
temp['passage_full'] = temp['passage_full'].apply(lambda x: x.replace('КонсультантПлюс надежная правовая поддержка', ''))
# temp.to_csv('путь куда сохранить consultants.csv', index=False)


add 44_FZ.pdf to dataframe
add 223_FZ.pdf to dataframe


In [ ]:
def combine_points(data):
    result = []
    current_article = ""
    current_section = ""

    for entry in data:
        if entry.startswith("Статья"):
            # Если строка начинается со слова "Статья", обновляем текущий заголовок
            current_article = entry
            current_section = ""
            result.append(current_article)
        elif entry.replace(".", "").isdigit():
            # Если это номер раздела (например, "1", "2"), обновляем текущий раздел
            current_section = f"{current_article}.{entry}"
            result.append(current_section)
        else:
            # Если это подпункт (например, "1)", "2)", "9.1)"), добавляем его к текущему разделу
            result.append(f"{current_section}.{entry}")

    return result

temp['new_number'] = combine_points(temp['number'])

In [80]:
from llama_index.core.node_parser import TokenTextSplitter
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('sergeyzh/rubert-tiny-turbo', cache_dir="../../models/")

def get_tokens(text, tokenizer=tokenizer):
    text_enc = tokenizer.encode(text)
    return text_enc

splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=50, tokenizer=get_tokens)

temp['last_passage'] = temp['passage_full'].apply(lambda x: splitter.split_text(x))

full_temp = temp.explode('last_passage').reset_index(drop=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (2625 > 2048). Running this sequence through the model will result in indexing errors


In [83]:
full_temp = full_temp.rename(columns={
    'last_passage': 'chunk'
})

full_temp.to_csv("../data/search.csv", index=False)

# Vector embedding Stores

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

model = HuggingFaceEmbedding("sergeyzh/rubert-tiny-turbo", cache_folder="../../models/rubert_turbo", device="cpu")

c:\Мисис\python_ml\rag_python_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import faiss
from llama_index.core import Document
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore
import sys
sys.path.append("../")
from config import settings

d = len(model.get_query_embedding('hello'))
faiss_index = faiss.IndexFlatL2(d)

In [22]:
import pandas as pd
import faiss
from llama_index.core import Document
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore

d = 312
faiss_index = faiss.IndexFlatL2(d)


class EmbeddingRetriever:
    def __init__(self, settings):
        # self.data = pd.read_csv(path_to_data)
        pass


    def make_docs(self):
        docs = []
        for ind, row in tqdm(self.data.iterrows(), total=len(self.data)):
            doc = Document(text=row['chunk'])
            doc.metadata = {
                "law_number": row['title'],
                "title": row['new_number'],
                "page_first": row['page_first'],
                "page_last": row['page_last'] 
            }
            docs.append(doc)
        return docs
    

    def make_and_save_index(self, faiss_index):
        vector_store = FaissVectorStore(faiss_index=faiss_index)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)
        index = VectorStoreIndex.from_documents(
            docs, storage_context=storage_context, show_progress=True, embed_model=model
        )
        index_folder = "../index/"
        index.storage_context.persist(index_folder)

        print(f'index Save to {index_folder}')

    
    def load_emb_index(self):
        vector_store = FaissVectorStore.from_persist_dir(settings.emb_index)
        storage_context = StorageContext.from_defaults(
            vector_store=vector_store, persist_dir=settings.emb_index
        )
        index = load_index_from_storage(storage_context=storage_context, embed_model=model)
        return index

In [23]:
emb_index = EmbeddingRetriever(settings).load_emb_index()

In [24]:
emb_index.as_retriever(similarity_top_k=20).retrieve('кто такой поставщик')

[NodeWithScore(node=TextNode(id_='e0b5624b-2a9e-4241-a2eb-31372ff90a93', embedding=None, metadata={'law_number': '44 Федеральный Закон', 'title': 'Статья 24.10.а)', 'page_first': 39, 'page_last': 39}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2d1ca8e9-f927-4273-8be7-94706e96e3ec', node_type='4', metadata={'law_number': '44 Федеральный Закон', 'title': 'Статья 24.10.а)', 'page_first': 39, 'page_last': 39}, hash='3be58450615c700075f9a329a92fc480ea2de6b54ee08a19e0d0ec64a8b54edb')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Статья 24. Способы определения поставщиков (подрядчиков, исполнителей) (в ред. Федерального закона  от 02.07.2021 N 360-ФЗ)\nа) закупки, по результатам которой заключается контракт на поставку товаров, необходимых для нормального жизнеобеспечения граждан; (пп. "а" в ред. Федерального закона  от 16.04.2022 N 104-ФЗ)', mimetype='text/plain', start_char_id

# BM25 Retriever

In [25]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer
from llama_index.core.schema import TextNode

In [26]:
from typing import List
from llama_index.core import Document
from llama_index.core.schema import TextNode
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer
from config import settings
import joblib


class BMRetriever:
    def __init__(self, settings):
        if settings.bm_index:
            self.nodes = joblib.load(settings.bm_index)

    def load_bm_retriever(self):
        bm25_retriever = BM25Retriever.from_defaults(
            nodes=self.nodes,
            similarity_top_k=settings.top_k_bm,
            stemmer=Stemmer.Stemmer("russian"),
            language="russian",
        )
        return bm25_retriever
    
    def make_nodes(self, docs: Document):
        nodes = []
        for doc in tqdm(docs):
            node = TextNode(text=doc.get_content())
            node.metadata = doc.metadata
            nodes.append(node)

        joblib.dump(nodes, settings.bm_index)
        print(f"Saved bm nodes to {settings.bm_index}")

In [27]:
bm_retriever = BMRetriever(settings).load_bm_retriever()

In [39]:
a = bm_retriever.retrieve('helllo')[0]

# FusionRetriever

In [1]:
from pydantic import BaseModel
from llama_index.core.schema import NodeWithScore

class FusionDTO(BaseModel):
    score: float
    law_number: str
    title: str
    page_first: int
    page_last: int
    text: str

    @classmethod
    def from_node_with_score(cls, node: NodeWithScore) -> "FusionDTO":
        """
        Create a FusionDTO instance from a NodeWithScore instance.

        Args:
            node (NodeWithScore): The source node object.

        Returns:
            FusionDTO: A new instance of FusionDTO.
        """
        return cls(
            score=node.score,
            law_number=node.metadata.get("law_number", ""),
            title=node.metadata.get("title", ""),
            page_first=node.metadata.get("page_first", 0),
            page_last=node.metadata.get("page_last", 0),
            text=node.text,
        )


In [ ]:
from llama_index.core.schema import NodeWithScore
from typing import List, Dict
from collections import defaultdict
import sys
sys.path.append("../")
from utils.retrievers.BM25Retriever import BMRetriever
from utils.retrievers.EmbeddingRetriever import EmbeddingRetriever
from config import settings

class FusionRetriever(BMRetriever, EmbeddingRetriever):
    def __init__(self, settings):
        BMRetriever.__init__(self, settings=settings)
        EmbeddingRetriever.__init__(self, settings=settings)

        self.bm25_r = self.load_bm_retriever()
        emb_index = self.load_emb_index()
        self.emb_r = emb_index.as_retriever(similarity_top_k=settings.top_k_emb)
    

    def normalize_scores(self, nodes: List[NodeWithScore]) -> List[FusionDTO]:
        """
        Преобразует список NodeWithScore в список FusionDTO и нормализует поле score по методу мин-макс.

        Аргументы:
            nodes (List[NodeWithScore]): Список объектов NodeWithScore для обработки.

        Возвращает:
            List[FusionDTO]: Список объектов FusionDTO с нормализованными значениями score.
        """
        dtos = [FusionDTO.from_node_with_score(node) for node in nodes]
        scores = [dto.score for dto in dtos]

        min_score = min(scores)
        max_score = max(scores)

        if max_score - min_score == 0:
            for dto in dtos:
                dto.score = 1.0  # Все значения одинаковы, нормализуем в 1.0
            return dtos

        for dto in dtos:
            dto.score = (dto.score - min_score) / (max_score - min_score)

        return dtos
    

    def rrf_fusion(self, list1: List[FusionDTO], list2: List[FusionDTO], k: int = 60, weights = settings.weights) -> List[FusionDTO]:
        """
        Выполняет RRF-фузию (Reciprocal Rank Fusion) для двух списков объектов FusionDTO, 
        учитывая ранги и веса скоров.

        Аргументы:
            list1 (List[FusionDTO]): Первый список объектов FusionDTO.
            list2 (List[FusionDTO]): Второй список объектов FusionDTO.
            k (int): Параметр RRF для контроля влияния ранга (по умолчанию: 60).
            weight1 (float): Вес первого списка (по умолчанию: 1.0).
            weight2 (float): Вес второго списка (по умолчанию: 1.0).

        Возвращает:
            List[FusionDTO]: Список объектов FusionDTO, объединённых по методу RRF, 
                            отсортированный по убыванию финальных RRF-оценок.
        """
        rrf_scores: Dict[str, float] = defaultdict(float)
        dto_mapping: Dict[str, FusionDTO] = {}

        def update_rrf_scores(dtos: List[FusionDTO], rank_offset: int, weight: float):
            """
            Обновляет RRF-оценки для объектов списка с учётом их текущего score.

            Аргументы:
                dtos (List[FusionDTO]): Список FusionDTO для обработки.
                rank_offset (int): Значение k, добавляемое к рангу.
                weight (float): Вес текущего списка.
            """
            for rank, dto in enumerate(dtos, start=1):
                identifier = f"{dto.law_number}-{dto.title}"
                adjusted_score = dto.score * weight
                rrf_scores[identifier] += adjusted_score / (rank + rank_offset)
                dto_mapping[identifier] = dto 

        update_rrf_scores(list1, k, weights[0])
        update_rrf_scores(list2, k, weights[1])

        combined = [
            (dto_mapping[identifier], score) for identifier, score in rrf_scores.items()
        ]
        combined.sort(key=lambda x: x[1], reverse=True)

        return [item[0] for item in combined]

        
    def retrieve(self, query):
        nodes_bm = self.bm25_r.retrieve(query)
        nodes_emb = self.emb_r.retrieve(query)

        dtos_bm = self.normalize_scores(nodes_bm)
        dtos_emb = self.normalize_scores(nodes_emb)

        return self.rrf_fusion(dtos_bm, dtos_emb)[:settings.fusion_top_k]

In [10]:
fusion_r = FusionRetriever(settings=settings)

In [12]:
a = fusion_r.retrieve('кто такой поставщик')

[(FusionDTO(score=0.954723532651644, law_number='44 Федеральный Закон', title='Статья 104', page_first=218, page_last=221, text='исключения информации, предусмотренной частью 3  настоящей статьи, из реестра недобросовестных поставщиков. (часть 10 в ред. Федерального закона  от 30.12.2020 N 539-ФЗ)\n11. Включение в реестр недобросовестных поставщиков информации об участнике закупки, о поставщике (подрядчике, исполнителе), содержащаяся в реестре недобросовестных поставщиков информация, неисполнение действий, предусмотренных частью 9  настоящей статьи, могут быть обжалованы заинтересованным лицом в судебном порядке. (в ред. Федерального закона  от 16.04.2022 N 104-ФЗ) Глава 6. ОБЖАЛОВАНИЕ ДЕЙСТВИЙ (БЕЗДЕЙСТВИЯ) СУБЪЕКТОВ КОНТРОЛЯ (в ред. Федерального закона  от 02.07.2021 N 360-ФЗ)'), 0.010487363755156366), (FusionDTO(score=1.0, law_number='44 Федеральный Закон', title='Статья 93.1.7)', page_first=157, page_last=157, text='Статья 93. Осуществление закупки у единственного поставщика (подря

In [15]:
a

[FusionDTO(score=0.954723532651644, law_number='44 Федеральный Закон', title='Статья 104', page_first=218, page_last=221, text='исключения информации, предусмотренной частью 3  настоящей статьи, из реестра недобросовестных поставщиков. (часть 10 в ред. Федерального закона  от 30.12.2020 N 539-ФЗ)\n11. Включение в реестр недобросовестных поставщиков информации об участнике закупки, о поставщике (подрядчике, исполнителе), содержащаяся в реестре недобросовестных поставщиков информация, неисполнение действий, предусмотренных частью 9  настоящей статьи, могут быть обжалованы заинтересованным лицом в судебном порядке. (в ред. Федерального закона  от 16.04.2022 N 104-ФЗ) Глава 6. ОБЖАЛОВАНИЕ ДЕЙСТВИЙ (БЕЗДЕЙСТВИЯ) СУБЪЕКТОВ КОНТРОЛЯ (в ред. Федерального закона  от 02.07.2021 N 360-ФЗ)'),
 FusionDTO(score=1.0, law_number='44 Федеральный Закон', title='Статья 93.1.7)', page_first=157, page_last=157, text='Статья 93. Осуществление закупки у единственного поставщика (подрядчика, исполнителя)\n7) 

In [ ]:
class Prompt:
    generation_prompt = """
        Ты ассистент поддержки в области государственных закупок по 44 и 223 Федеральным Законам.
        Твоя задача - ответить по предоставленным контекстам на вопрос пользователя. В случае, если в контексте недостаточно информации, попробуй найти ответ в своих знаниях.
        В том случае если твоих знаний и контектса не хватает для ответа ты должен вывести - Я не владею нужной информацией, обратитесь к консультанту.

        Пример1:
        Вопрос: Какие действия заказчик обязан предпринять перед проведением конкурса, и какой минимальный срок необходимо соблюдать?
        Ответ: Заказчик обязан разместить в единой информационной системе извещение о проведении конкурса и документацию о закупке. Это должно быть сделано не менее чем за пятнадцать дней до даты окончания срока подачи заявок на участие в конкурсе.

        Пример2:
        Вопрос: Какие требования предъявляются к электронным документам, передаваемым из корпоративных информационных систем в единую информационную систему?
        Ответ: Электронные документы, передаваемые из корпоративных информационных систем в единую информационную систему, должны быть подписаны электронной подписью.

        Вопрос: {question}
    """

In [19]:
few_shot = """
    Закон: {law}
    Номер статьи: {title}
    Страницы закона: {page_first} - {page_last}
    Контекст: {context}
"""

In [20]:
prompt = Prompt.generation_prompt

In [21]:
few_shot_prompts = [few_shot.format(
    law=elem.law_number, title=elem.title, page_first=elem.page_first, page_last=elem.page_last, context=elem.text
    ) for elem in a]

In [24]:
print("\n".join(few_shot_prompts))


    Закон: 44 Федеральный Закон
    Номер статьи: Статья 104
    Страницы закона: 218 - 221
    Контекст: исключения информации, предусмотренной частью 3  настоящей статьи, из реестра недобросовестных поставщиков. (часть 10 в ред. Федерального закона  от 30.12.2020 N 539-ФЗ)
11. Включение в реестр недобросовестных поставщиков информации об участнике закупки, о поставщике (подрядчике, исполнителе), содержащаяся в реестре недобросовестных поставщиков информация, неисполнение действий, предусмотренных частью 9  настоящей статьи, могут быть обжалованы заинтересованным лицом в судебном порядке. (в ред. Федерального закона  от 16.04.2022 N 104-ФЗ) Глава 6. ОБЖАЛОВАНИЕ ДЕЙСТВИЙ (БЕЗДЕЙСТВИЯ) СУБЪЕКТОВ КОНТРОЛЯ (в ред. Федерального закона  от 02.07.2021 N 360-ФЗ)


    Закон: 44 Федеральный Закон
    Номер статьи: Статья 93.1.7)
    Страницы закона: 157 - 157
    Контекст: Статья 93. Осуществление закупки у единственного поставщика (подрядчика, исполнителя)
7) заключение контракта на поставку